In [1]:
"""simple optimizer: finding minimum of y=x+2x^2"""

import tensorflow as tf
import numpy as np

a =tf.constant(1.0,name="a")
b=tf.constant(2.0,name="b")

x=tf.Variable(1.0,name="x")

out = tf.mul(a,x)+tf.mul(b,tf.mul(x,x))

out_summary=tf.scalar_summary("output_summary_optimization", out)
merged=tf.merge_all_summaries()

train_op = tf.train.GradientDescentOptimizer(0.001).minimize(out)

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.train.SummaryWriter('/tmp/tsb_log',sess.graph)

    for i in range(1000):
        sess.run(train_op)
        summary=sess.run(merged)
        writer.add_summary(summary,i)
        
    print("x:",sess.run(out))# x where is min
    print("y(x):",sess.run(x))# min y
    writer.close()

x: -0.123968
y(x): -0.227288


In [1]:
"""completely basic monolayer"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=15
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([10],stddev=0.01))

#model
def network(x,Weights1,Weights2,Biases1,Biases2):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    return layer_2


pred =network(Data,Weights_1,Weights_2,Biases_1,Biases_2)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))

cost_summary=tf.scalar_summary("cost_summary_monolayer", cost)

train_op = tf.train.GradientDescentOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.train.SummaryWriter('/tmp/tsb_log',sess.graph)
    for epoch in range(epoch_num):
        
        
        
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y})
            
        summary = sess.run(cost_summary,feed_dict={Data: batch_x, Labels: batch_y})
        writer.add_summary(summary, epoch)
        
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))
    writer.close()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy: 0.795


In [2]:
"""completely basic monolayer eval through test/train, not cost function"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=15
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([10],stddev=0.01))

#model
def network(x,Weights1,Weights2,Biases1,Biases2):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    return layer_2


pred =network(Data,Weights_1,Weights_2,Biases_1,Biases_2)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))



train_op = tf.train.GradientDescentOptimizer(training_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
accuracy_train_summary=tf.scalar_summary("accuracy_train_summary_monolayer", accuracy)
accuracy_test_summary=tf.scalar_summary("accuracy_test_summary_monolayer", accuracy)
init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.train.SummaryWriter('/tmp/tsb_log',sess.graph)
    for epoch in range(epoch_num):
        
        
        
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y})
        print ("Accuracy, test:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))
        summary_test = sess.run(accuracy_test_summary,feed_dict={Data: mnist.test.images, Labels:  mnist.test.labels})
        writer.add_summary(summary_test, epoch)
        summary_train = sess.run(accuracy_train_summary,feed_dict={Data: mnist.train.images, Labels:  mnist.train.labels})
        writer.add_summary(summary_train, epoch)        
    
    print ("Accuracy:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels}))
    writer.close()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy, test: 0.4201
Accuracy, test: 0.5648
Accuracy, test: 0.6039
Accuracy, test: 0.6188
Accuracy, test: 0.6365
Accuracy, test: 0.663
Accuracy, test: 0.6875
Accuracy, test: 0.7097
Accuracy, test: 0.7233
Accuracy, test: 0.7316
Accuracy, test: 0.7418
Accuracy, test: 0.7577
Accuracy, test: 0.775
Accuracy, test: 0.7888
Accuracy, test: 0.7981
Accuracy: 0.7981


In [1]:
"""completely basic multilayer -with dropout & Adam Optimizer"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=15
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
Weights_3=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_3=tf.Variable(tf.random_normal([10],stddev=0.01))



#dropout
keep_prob = tf.placeholder("float")


#model
def network(x,Weights1,Weights2,Weights3,Biases1,Biases2,Biases3):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu6(layer_1)#possible instead: sigmoid (80% acc), relu (90-96 % accuracy) 
 
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    layer_2=tf.nn.relu6(layer_2)
    
    drop = tf.nn.dropout(layer_2, keep_prob)
    
    layer_3=tf.add(tf.matmul(drop,Weights3),Biases3)
    return layer_3


pred =network(Data,Weights_1,Weights_2,Weights_3,Biases_1,Biases_2,Biases_3)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))
cost_summary=tf.scalar_summary("cost_summary_expanded", cost)


train_op = tf.train.AdamOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    writer = tf.train.SummaryWriter('/tmp/tsb_log',sess.graph)
    
    for epoch in range(epoch_num):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y,keep_prob: 1.0 })
            
        summary = sess.run(cost_summary,feed_dict={Data: batch_x, Labels: batch_y,keep_prob: 1.0 })
        writer.add_summary(summary, epoch)
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    print ("Accuracy-test:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels,keep_prob: 1.0}))
    print ("Accuracy-train:", accuracy.eval({Data: mnist.train.images, Labels: mnist.train.labels,keep_prob: 1.0}))
    print ("Accuracy-eval:", accuracy.eval({Data: mnist.validation.images, Labels: mnist.validation.labels,keep_prob: 1.0}))
    writer.close()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy-test: 0.9774
Accuracy-train: 0.995818
Accuracy-eval: 0.98


In [1]:
"""completely basic multilayer -with dropout & Adam Optimizer- tensorboard accuracy evaluation"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

#numbers:
epoch_num=15
training_rate=0.001
batch_size=100

#placeholders & variables
Data=tf.placeholder("float32",[None,784])
Labels=tf.placeholder("float32",[None,10])

Weights_1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
Weights_2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
Weights_3=tf.Variable(tf.random_normal([256,10],stddev=0.01))

Biases_1=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_2=tf.Variable(tf.random_normal([256],stddev=0.01))
Biases_3=tf.Variable(tf.random_normal([10],stddev=0.01))



#dropout
keep_prob = tf.placeholder("float")


#model
def network(x,Weights1,Weights2,Weights3,Biases1,Biases2,Biases3):
    layer_1=tf.add(tf.matmul(x,Weights1),Biases1)
    layer_1=tf.nn.relu6(layer_1)#possible instead: sigmoid (80% acc), relu (90-96 % accuracy) 
 
    layer_2=tf.add(tf.matmul(layer_1,Weights2),Biases2)
    layer_2=tf.nn.relu6(layer_2)
    
    drop = tf.nn.dropout(layer_2, keep_prob)
    
    layer_3=tf.add(tf.matmul(drop,Weights3),Biases3)
    return layer_3


pred =network(Data,Weights_1,Weights_2,Weights_3,Biases_1,Biases_2,Biases_3)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Labels))

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
accuracy_train_summary=tf.scalar_summary("accuracy_train_summary_multilayer", accuracy)
accuracy_test_summary=tf.scalar_summary("accuracy_test_summary_mulitlayer", accuracy)

train_op = tf.train.AdamOptimizer(training_rate).minimize(cost)

init =tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    writer = tf.train.SummaryWriter('/tmp/tsb_log',sess.graph)
    
    for epoch in range(epoch_num):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run([train_op, cost], feed_dict={Data: batch_x, Labels: batch_y,keep_prob: 1.0 })
            
        print ("Accuracy, test:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels,keep_prob: 1.0}))
        summary_test = sess.run(accuracy_test_summary,feed_dict={Data: mnist.test.images, Labels:  mnist.test.labels,keep_prob: 1.0})
        writer.add_summary(summary_test, epoch)
        summary_train = sess.run(accuracy_train_summary,feed_dict={Data: mnist.train.images, Labels:  mnist.train.labels,keep_prob: 1.0})
        writer.add_summary(summary_train, epoch)
    
  

    print ("Accuracy-test:", accuracy.eval({Data: mnist.test.images, Labels: mnist.test.labels,keep_prob: 1.0}))
    print ("Accuracy-train:", accuracy.eval({Data: mnist.train.images, Labels: mnist.train.labels,keep_prob: 1.0}))
    print ("Accuracy-eval:", accuracy.eval({Data: mnist.validation.images, Labels: mnist.validation.labels,keep_prob: 1.0}))
    writer.close()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy, test: 0.9394
Accuracy, test: 0.9659
Accuracy, test: 0.9707
Accuracy, test: 0.9736
Accuracy, test: 0.9759
Accuracy, test: 0.9761
Accuracy, test: 0.9781
Accuracy, test: 0.9799
Accuracy, test: 0.9788
Accuracy, test: 0.9776
Accuracy, test: 0.9794
Accuracy, test: 0.9782
Accuracy, test: 0.9788
Accuracy, test: 0.977
Accuracy, test: 0.9807
Accuracy-test: 0.9807
Accuracy-train: 0.998564
Accuracy-eval: 0.9824
